In [4]:
#!/usr/bin/python

from apiclient.discovery import build
# from apiclient.errors import HttpError
# from oauth2client.tools import argparser

In [5]:
import json
with open('secret.json') as f:
    secret = json.load(f)

In [10]:
import pandas as pd

### 関数化

In [6]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, 
                        developerKey=DEVELOPER_KEY)

In [7]:
def video_search(youtube, q='自動化', max_results=50):
    response = youtube.search().list(
        q=q,
        part="id,snippet",
        order='viewCount',
        type='video',
        maxResults=max_results
      ).execute()
    
    items_id = []
    items = response['items']
    for item in items:
        item_id = {}
        item_id['video_id'] = item['id']['videoId']
        item_id['channel_id'] = item['snippet']['channelId']
        items_id.append(item_id)

    df_video = pd.DataFrame(items_id)
    
    return df_video

In [18]:
def get_results(df_video, threshold=5000):
    channel_ids = df_video['channel_id'].unique().tolist()

    subscriber_list = youtube.channels().list(
        part="statistics",
        id=','.join(channel_ids),
        fields='items(id , statistics(subscriberCount))'
      ).execute()

    subscribers = []
    for item in subscriber_list['items']:
        subscriber = {}
        if len(item['statistics']) > 0:
            subscriber['channel_id'] = item['id']
            subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
        else:
            subscriber['channel_id'] = item['id']
        subscribers.append(subscriber)

    df_subscribers = pd.DataFrame(subscribers)

    df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')

    df_extracted = df[df['subscriber_count'] < 10000]

    video_ids = df_extracted['video_id'].tolist()

    videos_list = youtube.videos().list(
        part="snippet, statistics",
        id=','.join(video_ids),
        fields='items(id , snippet(title), statistics(viewCount))'
      ).execute()

    items = videos_list['items']
    videos_info = []
    for item in items:
        video_info = {}
        video_info['video_id'] = item['id']
        video_info['title'] = item['snippet']['title']
        video_info['view_count'] = item['statistics']['viewCount']
        videos_info.append(video_info)

    df_videos_info = pd.DataFrame(videos_info)

    results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')

    results = results.loc[:, ['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]
    
    return results

In [19]:
df_video = video_search(youtube, q='Python Excel', max_results=50)
results = get_results(df_video, threshold=5000)

In [20]:
results

,video_id,title,view_count,subscriber_count,channel_id
0,ubqsRcCUcB4,Use Python to replace VBA in Excel,327469,8890.0,UCfMaA6FC0aKYXEWubmOW5Vw
1,p0DNcTnreuY,How to read Excel files with Python (xlrd tuto...,256023,2150.0,UCWvcnspqDKqOaalGA-wxO9w
2,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),233377,6290.0,UCbg0ptFI3yZ4Pe5vmUiEzzA
3,C2O3O_GydV4,Use Python and Pandas to Work With Excel,64112,485.0,UCAOvckbaZM8MEh9R83KH9HA
4,JwZtTk6Hr80,How to Connect Python to Ms Excel Read and Wri...,55031,6990.0,UCvHgEAcw6VpcOA3864pSr5A
5,JfU1G1Ug6-k,BeautifulSoup 4 Python Web Scaping to CSV Exce...,46523,7210.0,UCblQ2Ah7dDm-yql4Z1K1E0A
6,S5EVZwXnleM,Python Load Excel XLSX into variables,43708,2600.0,UC4kU17BqNXaCKGFAYvrlawg
7,8rfH7al7PgU,Import excel file in python Jupyter Notebook| ...,40816,628.0,UCiUvYbjNotTke1B7QgeDe1A
